# CML APIv2 を使った自動でのアプリのデプロイ
Pinecone に必要な文脈情報を追加し、準備ができた状態で、このノートブックの内容を実行してください。

#### 4.1 必要なライブラリのインポート

In [ ]:
import os
import cmlapi
import random
import string
import json

#### 4.2 CML API クライアントを生成し、利用可能なランタイムの一覧を取得する
このコードでは、CML（Cloudera AI）の環境に接続し、まず以下の条件に該当するランタイムを取得します。

- Python 3.10 のランタイムである
- Nvidia の GPU をサポートしている
- エディタとして Jupyter Lab を利用できる
  
その上で、リストの中にある最新のランタイムを選択し、内容を一覧表示するとともに、将来のジョブ実行時に利用できるよう、ランタイムの識別子を環境変数に保存します。

In [ ]:
client = cmlapi.default_client(url=os.getenv("CDSW_API_URL").replace("/api/v1", ""), cml_api_key=os.getenv("CDSW_APIV2_KEY"))
available_runtimes = client.list_runtimes(search_filter=json.dumps({
    "kernel": "Python 3.10",
    "edition": "Nvidia GPU",
    "editor": "JupyterLab"
}))
print(available_runtimes)

## 利用可能なランタイムを、環境の中の最新のランタイムとする（イテレータは0から始まり、1ずつ増えていきます）
## JOB_IMAGE_ML_RUNTIME は、variable stores the ML Runtime which will be used to launch the job
print(available_runtimes.runtimes[-1])
print(available_runtimes.runtimes[-1].image_identifier)
APP_IMAGE_ML_RUNTIME = available_runtimes.runtimes[-1].image_identifier

## 将来のジョブ実行のために、MLのランタイムを環境変数に保存しておく
os.environ['APP_IMAGE_ML_RUNTIME'] = APP_IMAGE_ML_RUNTIME

#### 4.3 現在のプロジェクトを取得する
環境変数の "CDSW Project ID" から現在のプロジェクトの情報を取得し、メタデータを表示します。

In [ ]:
project = client.get_project(project_id=os.getenv("CDSW_PROJECT_ID"))
print(project)

#### 4.4 LLM アプリケーションを作成する

このコードでは、"CML - Auto created"という名前のCMLアプリケーションを作成します。  
プロジェクトの description や、リソースの割り当て（CPU、メモリ）についても記述しています。  
ランタイムの識別子は、これまでの流れで取得したものです。

In [ ]:
application_request = cmlapi.CreateApplicationRequest(
     name = "CML - Auto created",
     description = "このアプリケーションは、4_launch_hosted_app配下の create_llm_application.ipynb を使って llm_prototype_app.py を呼び出すことで作成されたものです。UI部分には Gradio UIを使用しています。",
     project_id = project.id,
     subdomain = "cml-llm-interface",
     script = "4_launch_hosted_app/llm_prototype_app.py",
     cpu = 2,
     memory = 8,
     runtime_identifier = os.getenv('APP_IMAGE_ML_RUNTIME')
)

app = client.create_application(
     project_id = project.id,
     body = application_request
)